# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [6]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [7]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [8]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [9]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [10]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.3-70b-versatile"

## Creating Crew, Agents, and Tasks

In [11]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2026-02-09 01:39:24,124 - 140097324432256 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


## Kicking off the Crew

In [12]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: To create a detailed summary report highlighting the most impactful financial news and updates related to Inflation in the US and the impact on the stock market in 2024, I need to gather the latest information on this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation news 2024 stock market impact\"}"
## Tool Outp



# Agent: Lead Market Analyst
## Final Answer: 
Market Update: US Stocks Rise on Strong Data and Low Inflation

The US stock market has risen on strong data and low inflation, with the Dow Jones Industrial Average gaining 133.37 points, or 0.4%, to close at 35,125.17, while the S&P 500 Index advanced 23.57 points, or 0.6%, to 4,061.18.

The current inflation rate in the US is 2.7%, which is below the Federal Reserve's target of 2%. The core CPI, which excludes food and energy costs, has remained flat at 2.6% over the past month.

The Federal Reserve has cut interest rates by 1.0% in late 2024 and 0.75% in late 2025, and is expected to cut rates further in 2026. The Fed's median projections call for core PCE to fall to 2.5% by the end of 2026.

Investors are watching inflation closely, as it can have a significant impact on their personal budget and investment decisions. The current environment is changing quickly, especially with the evolving inflation outlook and potential impact of 



# Agent: Chief Data Strategist
## Thought: Thought: To provide a comprehensive analysis report that highlights key market trends and actionable insights related to Inflation in the US and the impact on the stock market in 2024, I need to utilize the available tools to gather relevant information and data.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation rate 2024 and stock market impact\"}"
## Tool Output: 

Search results: Title: Analysis: Assessing Inflation's Impact | U.S. Bank
Link: https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html
Snippet: After rapidly raising interest rates in 2022 and 2023 to fight inflation, the Fed cut rates by 1.0% in late 2024 and 0.75% in late 2025. The Fed ...
---
Title: Current U.S. Inflation Rate is 2.7%: Chart and Why It Matters
Link: https://www.nerdwallet.com/investing/learn/inflation
Snippet: The current U.S. inflation rate is 2.7%. The CPI ros

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]




# Agent: Chief Data Strategist
## Thought: Title: Impact of Inflation on the US Stock Market: An Empirical Study ...
Link: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4398404
Snippet: The study investigates the relationship between inflation and stock market returns in the US from 1960 to 2024.
Thought: Based on the search results, I have gathered relevant information and data on the current US inflation rate, its impact on the stock market, and the Federal Reserve's actions. I can now use this information to provide a comprehensive analysis report.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"inflation impact on stock market\", \"website\": \"https://www.investopedia.com/\"}"
## Tool Output: 
Relevant Content:
real GDP forecasted to range between 1.6% and 1.9% per year. Persistent inflation and geopolitical conflicts (20%): While our baseline remains positive, there are always downside risks to any forecast. We see risks centering around 



# Agent: Creative Content Director
## Thought: I need to gather more insights on the current inflation situation in the US and its impact on the stock market in 2024 to create high-quality content tailored for investors looking to understand these dynamics better.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation 2024 impact on stock market\"}"
## Tool Output: 

Search results: Title: How Do Changing Interest Rates Affect the Stock Market? - U.S. Bank
Link: https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html
Snippet: The Federal Reserve kept rates at 3.50%–3.75%, noting improving inflation and labor trends as investors continue to price in two cuts for 2026.
---
Title: [PDF] How Inflation Affects Your Stock Market Returns
Link: https://pattersonandrews.nm.com/mediahandler/media/675996/pawm-how-inflation-affects.pdf
Snippet: Thankfully, inflation has been increasing at a slowe



# Agent: Chief Content Officer
## Final Answer: 
```markdown
# Navigating Inflation's Impact on the US Stock Market in 2024

As we step into 2024, the US stock market finds itself at a crossroads shaped by a shifting inflation landscape. Investors are keenly observing inflation rates, which currently sit at **2.7%**, just above the Federal Reserve's target of **2%**. This slight inflationary pressure is significant, impacting everything from consumer spending to Federal Reserve monetary policy.

## Current Inflation Context

Recent economic indicators show that inflation has been gradually easing, arriving at a **2.7%** annual rate. This follows a pattern observed in late 2024 when the Federal Reserve began cutting interest rates to combat persistent price pressures. The reduction by **1.0%** in late 2024 and **0.75%** in late 2025 has provided a cushion for stock market performance, allowing for a modest recovery in consumer purchasing power. 

The easing inflationary figure, althou

## Social Content

In [13]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
📈 Inflation sits at 2.7% as we enter 2024,
slightly above the Fed's target. Is this impacting
your investment strategy? Keep an eye on the
market trends! 💰 #InvestSmart #Inflation2024
--------------------------------------------------
LinkedIn
As inflation trends toward 2.7%, the stock market
is showing resilience with significant gains in
major indices. Join us in exploring effective
investment strategies for navigating inflation in
2024! #InvestmentStrategy #USStockMarket
--------------------------------------------------
Facebook
Navigating 2024's stock market amidst inflationary
pressures can be tricky! Tune in as we discuss
market trends and how to adjust your investment
strategies to adapt. Your financial health is a
priority! 💹 #MarketInsights #FinancialGoals
--------------------------------------------------


## Blog Post

In [14]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Navigating Inflation's Impact on the US Stock Market in 2024

As we step into 2024, the US stock market finds itself at a crossroads shaped by a shifting inflation landscape. Investors are keenly observing inflation rates, which currently sit at **2.7%**, just above the Federal Reserve's target of **2%**. This slight inflationary pressure is significant, impacting everything from consumer spending to Federal Reserve monetary policy.

## Current Inflation Context

Recent economic indicators show that inflation has been gradually easing, arriving at a **2.7%** annual rate. This follows a pattern observed in late 2024 when the Federal Reserve began cutting interest rates to combat persistent price pressures. The reduction by **1.0%** in late 2024 and **0.75%** in late 2025 has provided a cushion for stock market performance, allowing for a modest recovery in consumer purchasing power. 

The easing inflationary figure, although slightly above the desired target, offers a glimmer of hope for both the economy and investors. A stable inflation rate is crucial for fostering investor confidence and stimulating spending habits among consumers. As potential buyers regain their confidence, it could bolster economic growth and further enhance stock market stability.

## Stock Market Response

The stock market's resilience has been evident in the upward trends of major indices, with the **Dow Jones gaining 133.37 points** and the **S&P 500 advancing by 23.57 points** just recently. This surge reflects investor confidence in the potential for stable growth, even amid fluctuating inflation. Particularly, the S&P 500 has reportedly reached record highs multiple times throughout 2024, signaling strong performance driven by optimism regarding corporate earnings and consumer spending.

Investors should monitor these indices closely as they reflect market sentiment and investor behavior. The trends seen thus far indicate that the stock market is reacting positively to perceived stability, and this could invite more investment, which in turn could promote even further growth. It's essential for investors to align their strategies with this optimistic outlook while remaining alert to the developments in inflation rates.

## The Connection Between Inflation and Investment Decisions

Investors are becoming increasingly aware of how inflation influences their investment strategies. High inflation can erode purchasing power and lead to increased costs for businesses, which in turn affects profit margins. As we reflect on the past year, it’s clear that well-planned strategies focusing on diversification have helped investors navigate this volatile landscape. 

Maintaining a diversified portfolio, including inflation-sensitive assets like equities and bonds, remains critical. Investors should consider allocating resources to sectors that historically perform well during inflationary periods. Additionally, incorporating commodities and real estate into a diversified portfolio can shield against the adverse effects of inflation, offering a hedge against rising prices.

## Future Outlook

Moving forward, the Federal Reserve's commitment to addressing inflation through adjusted interest rates will be consequential. Projections indicate that **core PCE (Personal Consumption Expenditures)** could potentially drop to **2.5%** by the end of 2026, which, if achieved, could stabilize the market further. As a result, a consistent long-term investment strategy, paired with adaptive risk management, will empower investors to weather upcoming economic fluctuations.

The evolving landscape demands that investors remain agile, adapting their investment approaches in response to changes in the economic environment. Continuous education and awareness of market shifts will enable investors to capitalize on opportunities while mitigating risks associated with inflation.

## Conclusion

In conclusion, 2024 presents unique challenges and opportunities shaped by the ongoing inflation discourse. Investors who remain informed and adaptable will harness the potential to thrive in this evolving economic environment. By carefully navigating their investment choices and staying attuned to the Fed’s actions, the pathway to achieving financial goals is more attainable than ever.